## Python을 이용한 Redshift Serverless 사용
---

### 패키지 설치

In [1]:
# AWS Pandas SDK 를 설치합니다.
!pip install awswrangler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 2.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [1]:
# Redshift Connector 를 설치합니다.
!pip install redshift_connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
  Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 5.8 MB/s eta 0:00:00
Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl (8.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.6 MB/s eta 0:00:00


In [3]:
#  환경변수 설정파일을 통해서 연결정보를 로드하기 위해서 Python Dotenv 를 설치합니다.
!pip install python_dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


### 라이브러리 및 환경변수 로드

In [6]:
import pandas as pd
import redshift_connector
import os
import awswrangler as wr
from dotenv import load_dotenv
# 환경설정 파일(.env)에서 환경변수를 로드합니다.
# 환견변수 설정파일(.env)에서 환경변수를 로드 하지 않고 직접 셀에 입력할수 도 있지만 보안상 좋지 못합니다.
load_dotenv(".env")

False

### DB 연결 생성

In [2]:
# 연결 생성
conn = redshift_connector.connect(
     host=os.environ['REDSHIFT_HOST'],
     port=os.environ['REDSHIFT_PORT'],
     database=os.environ['REDSHIFT_DB'],
     user=os.environ['REDSHIFT_USER'],
     password=os.environ['REDSHIFT_PASSWORD'],
)

### 데이터 조회

In [5]:
wr.redshift.read_sql_query("SELECT * FROM datamart.houseprice LIMIT 5", conn)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,2660000,2800,3,1,1,yes,no,no,no,no,0,no,unfurnished
1,2660000,2430,3,1,1,no,no,no,no,no,0,no,unfurnished
2,2660000,3480,2,1,1,yes,no,no,no,no,1,no,semi-furnished
3,2660000,4000,3,1,1,yes,no,no,no,no,0,no,semi-furnished
4,2653000,3185,2,1,1,yes,no,no,no,yes,0,no,unfurnished


### 데이터 삽입

In [8]:
# 데이터프레임 생성 (예시 데이터)
data = {
    'price': [2660000, 2653000],
    'area': [2800, 3185],
    'bedrooms': [3, 2],
    'bathrooms': [1, 1],
    'stories': [1, 1],
    'mainroad': ['yes', 'yes'],
    'guestroom': ['no', 'no'],
    'basement': ['no', 'no'],
    'hotwaterheating': ['no', 'no'],
    'airconditioning': ['no', 'yes'],
    'parking': [0, 0],
    'prefarea': ['no', 'no'],
    'furnishingstatus': ['unfurnished', 'unfurnished']
}

df = pd.DataFrame(data)

# Redshift Serverless에 데이터 삽입
wr.redshift.to_sql(
    df,
    schema='datamart',
    table='houseprice',
    con=conn,
    mode='append',  # 데이터 추가 모드
    index=False
)

In [11]:
# 추가된 데이터 확인을 위한 쿼리 실행
wr.redshift.read_sql_query("SELECT * FROM datamart.houseprice where area in (2800, 3185)", conn)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4956000,2800,3,2,2,no,no,yes,no,yes,1,no,semi-furnished
1,2660000,2800,3,1,1,yes,no,no,no,no,0,no,unfurnished
2,2653000,3185,2,1,1,yes,no,no,no,yes,0,no,unfurnished
3,3430000,3185,2,1,1,yes,no,no,no,no,2,no,unfurnished
4,3360000,3185,2,1,1,yes,no,yes,no,no,2,no,furnished
5,2660000,2800,3,1,1,yes,no,no,no,no,0,no,unfurnished
6,2653000,3185,2,1,1,yes,no,no,no,yes,0,no,unfurnished


### 데이터 삭제

In [19]:
# 데이터 삭제 쿼리 실행
cursor = conn.cursor()
cursor.execute("DELETE FROM datamart.houseprice WHERE area IN (2800, 3185)")

In [20]:
wr.redshift.read_sql_query("SELECT * FROM datamart.houseprice where area in (2800, 3185)", conn)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus


In [ ]:
conn.commit()
cursor.close()
conn.close()